## Vicuna

### 1. Requirements

In [1]:
!pip3 install fschat

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 26.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 39.5 MB/s eta 0:00:0

### 2. LLaMA weigth 다운로드

30GB RAM에서 작동할 수 있도록 Vicuna-7B를 이용한다. Vicuna-7B의 경우 llama-7b를 이용하여 vicuna weight를 추가하는 방법으로 얻을 수 있다.

LLaMA weights:
https://huggingface.co/decapoda-research


In [3]:
## LLaMA weight 다운로드
!git lfs install
!git clone git@hf.co:decapoda-research/llama-7b-hf


git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
Cloning into 'llama-7b-hf'...
The authenticity of host 'hf.co (3.210.66.237)' can't be established.
ECDSA key fingerprint is SHA256:aBG5R7IomF4BSsx/h6tNAUVLhEkkaNGB8Sluyh/Q/qY.
Are you sure you want to continue connecting (yes/no/[fingerprint])? 

In [ ]:
"""
tokenizer_config.json 수정

"tokenizer_class": "LLaMATokenizer" -> "tokenizer_class": "LlaMATokenizer"
"""

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./llama-7b-hf")

model = AutoModelForCausalLM.from_pretrained("./llama-7b-hf")

/home/zai-user/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Tokenizer class LLaMATokenizer does not exist or is not currently imported.